In [4]:
import dlisio
import pandas as pd
import numpy as np
import cv2

In [5]:
#FMS
def open_fms(filename:str) -> pd.DataFrame:
    f, *f_tail = dlisio.load(filename)
    if len(f_tail): logging.warning('There are more logical files in tail')
    
    frame =f.object('FRAME','1B').curves()
    
    df = pd.DataFrame(frame)
    df["TDEP"]=df["TDEP"]* 0.00254
    df.drop(["FRAMENO"], axis=1, inplace=True)
    
    #df = normalize_values(df)
    
    
   # cols_list = [['TDEP','BA11', 'BA12', 'BA13', 'BA14', 'BA15', 'BA16', 'BA17', 'BA18', 
   #          'BA21','BA22', 'BA23', 'BA24', 'BA25', 'BA26', 'BA27', 'BA28'],
   #         ['TDEP','BB11', 'BB12', 'BB13', 'BB14', 'BB15', 'BB16', 'BB17', 'BB18', 
   #          'BB21','BB22', 'BB23', 'BB24', 'BB25', 'BB26', 'BB27', 'BB28'],
   #         ['TDEP','BC11', 'BC12', 'BC13', 'BC14', 'BC15', 'BC16', 'BC17', 'BC18', 
   #          'BC21','BC22', 'BC23', 'BC24', 'BC25', 'BC26', 'BC27', 'BC28']]

    
    #pads = [df[cols].copy() for cols in cols_list]
    #pads_norm = [normalize_values(pad) for pad in pads]
    #pads_concatenated = [concatenate_normalized(pad) for pad in pads_norm]  
    return normalize_values(df)

def normalize_values(data:pd.DataFrame) -> pd.DataFrame:
    TDEP= data['TDEP'].copy()
    data.drop(['TDEP'], axis=1, inplace=True)
    MAX_FMS_VALUE = 100
    data = (data-data.min())/(MAX_FMS_VALUE-data.min())*255
    data['TDEP'] = TDEP        
    return data

def concatenate_normalized(data:pd.DataFrame) -> pd.DataFrame:
    print(data.columns)
    fms1df = data[['BA11', 'BA12', 'BA13', 'BA14', 'BA15', 'BA16', 'BA17', 'BA18', 
                   'BA21','BA22', 'BA23', 'BA24', 'BA25', 'BA26', 'BA27', 'BA28']].copy()
    fms2df = data[['BB11', 'BB12', 'BB13', 'BB14', 'BB15', 'BB16', 'BB17', 'BB18', 
                   'BB21','BB22', 'BB23', 'BB24', 'BB25', 'BB26', 'BB27', 'BB28']].copy()
    fms3df = data[['BC11', 'BC12', 'BC13', 'BC14', 'BC15', 'BC16', 'BC17', 'BC18', 
                   'BC21','BC22', 'BC23', 'BC24', 'BC25', 'BC26', 'BC27', 'BC28']].copy()
    fms123df = pd.concat([fms1df, fms2df, fms3df], axis=1, join="inner")
    return data

def core_info(filename:str)-> pd.DataFrame:
    data = pd.read_csv(filename)
    data['max_depth'] = data['min_depth']+data['core_v_pixel']/20000
    print((data['core_v_pixel']-data['core_v_pixel']%512)/512)
    return data

def FMS_to_core(data:pd.DataFrame):
    FMS_list= []
    for index, interval in core_info.iterrows():
        min_depth = core_info.min_depth
        max_depth = core_info.max_depth
        FMS_coredepth = (open_fms[(open_fms['TDEP']>=min_depth) & (open_fms['TDEP']<=max_depth)])
        FMS_nparray=create_FMS_image(FMS_coredepth)
        FMS_list.append(FMS_nparray)
    return FMS_list

def create_FMS_image(FMS_list):
    FMS123 = FMS_list(["BA11","BB11","BC11","BA12","BB12","BC12",
                       "BA13","BB13","BC13","BA14","BB14","BC14",
                       "BA15","BB15","BC15","BA16","BB16","BC16",
                       "BA17","BB17","BC17","BA18","BB18","BC18",
                       "BA21","BB21","BC21","BA22","BB22","BC22",
                       "BA23","BB23","BC23","BA24","BB24","BC24",
                       "BA25","BB25","BC25","BA26","BB26","BC26",
                       "BA27","BB27","BC27","BA28","BB28","BC28"]) 
    FMS123_array = FMS123.values
    FMS123_reshaped = FMS123_array.reshape(len(FMS123),16,3).astype(np.uint8)
    print(FMS123_reshaped.shape)
    return FMS123_reshaped

def resize_FMS_image(img, dim, interpolation=cv2.INTER_LANCZOS4):
    width = 512
    height = core_info['core_v_pixel']
    dim = (width, height)
    FMS= cv2.resize(FMS_img, dim, interpolation)
    return image

def save_FMS_image(img):
    cv2.imwrite()
    

def slice_FMS(img, overlap=0.5):
    slice_dim = 512
    stride = int((1-overlap)*slice_dim)
    FMS_images = int(img.shape[0]/stride)-1
    
    FMS_slices= []
    for i in range(FMS_images):
        slice = img[i:i+slice_dim, :]
        FMS_slices.append(slice)
        name = f'{filename}_splitted'
        frmt = 'jpg'
        cv2.imwrite('{}_{}.{}'.format(name, i, frmt), slice)
        
        print(f'Succesfully created {FMS_images} images')
        return FMS_slices

In [6]:
#CORE
def crop_slice_image(filename:str):
    '''crops the core images width to 512pixel from mid +-256pixels'''
    img = cv2.imread(filename)
    img_h, img_w, _ = img.shape
    w1, h1 = int(img_w/2)-256, int(img_h)
    cropped_image = img[w1:w1+512, 0:h1]
    if cropped_image.shape[0]<512:
        return [] # return an empty list if the image is not big enough. Otherwise continues the code below...

    sliced_image = [slice_image(cropped_image,filename) for cropped_image in img]
    return sliced_image

def slice_image(img, filename, overlap=0.5):
    '''slice the core images to 512x512 pixel and overlaps images by 50%.
    Notice that I take the cropped image as my input to the function'''
    slice_dim = 512
    stride = int((1-overlap)*slice_dim) #The stride in pixel is simply the overlap time the pixel dimentions
    nb_images = int(img.shape[0]/stride)-1
    #The total number of image is equal to how many full strides you can fit minus 1!
    
    # Now let's do a loop between 0 and the total number of images, and save the images:
    slices = []
    
    for i in range(nb_images):
        for j in range (nb_images):
            slice = img[i:i+slice_dim, :]
            slices.append(slice)
            name = f'{filename}_splitted'
            frmt = 'jpg'
            cv2.imwrite('{}_{}.{}'.format(name, i, frmt), slice)
        
    print(f'Succesfully created {nb_images} images')
    return slices


In [8]:
open_fms('../raw_data/MESTB _009_1B.dlis')

KeyError: "None of [Index(['BB11', 'BB12', 'BB13', 'BB14', 'BB15', 'BB16', 'BB17', 'BB18', 'BB21',\n       'BB22', 'BB23', 'BB24', 'BB25', 'BB26', 'BB27', 'BB28'],\n      dtype='object')] are in the [columns]"

In [ ]:
core_info('../raw_data/1196A_coreinfo.csv')

In [ ]:
filename = '../raw_data/ 194-1196A-10R-1_7.511.tif'
slices = crop_slice_image(filename)